In [ ]:
from dataset import MysqlIO
import pandas as pd
import pandas_profiling as pp
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

In [ ]:
db = MysqlIO(host='relational.fit.cvut.cz', 
            database='financial', 
            user='guest', 
            password='relational')

In [ ]:
db.execute('SHOW TABLES')

In [ ]:
loan_df = db.execute_to_df('SELECT * FROM loan')
order_df = db.execute_to_df('SELECT * FROM `order`')
trans_df = db.execute_to_df('SELECT * FROM trans')
disp_df = db.execute_to_df('SELECT * FROM disp')
account_df = db.execute_to_df('SELECT * FROM account')
client_df = db.execute_to_df('SELECT * FROM client')
card_df = db.execute_to_df('SELECT * FROM card')
district_df = db.execute_to_df('SELECT * FROM district')

In [ ]:
def summary_df(df, table_name=''):
    print(table_name)
    print(df.shape)
    display(df.head())
    print('missing value')
    display(df.isnull().sum())

In [ ]:
summary_df(loan_df, "loan")
summary_df(order_df, "order")
summary_df(trans_df, "trans")
summary_df(disp_df, "disp")
summary_df(account_df, "account")
summary_df(client_df, "client")
summary_df(card_df, "card")
summary_df(district_df, "district")

1 client can have multiple accounts.
Also, 1 account can have multiple clients (owner, desponent).

In [ ]:
print(loan_df['account_id'].is_unique)
print(account_df['account_id'].is_unique)
print(disp_df['account_id'].is_unique)

# Data manipulation

In [ ]:
def add_prefix_to_colnames(df, prefix, fixed_col_name='account_id'):
    df = df.add_prefix(prefix)
    df = df.rename(index=str, columns={prefix+fixed_col_name : fixed_col_name})
    return df

In [70]:
loan_df_ = add_prefix_to_colnames(loan_df, prefix='loan_')
print(loan_df_.shape)
loan_df_.head()

(682, 7)


,loan_loan_id,account_id,loan_date,loan_amount,loan_duration,loan_payments,loan_status
0,4959,2,1994-01-05,80952,24,3373.00,A
1,4961,19,1996-04-29,30276,12,2523.00,B
2,4962,25,1997-12-08,30276,12,2523.00,A
3,4967,37,1998-10-14,318480,60,5308.00,D
4,4968,38,1998-04-19,110736,48,2307.00,C


In [71]:
account_df_ = add_prefix_to_colnames(account_df, prefix='acc_')
print(account_df_.shape)
account_df_.head()

(4500, 4)


,account_id,acc_district_id,acc_frequency,acc_date
0,1,18,POPLATEK MESICNE,1995-03-24
1,2,1,POPLATEK MESICNE,1993-02-26
2,3,5,POPLATEK MESICNE,1997-07-07
3,4,12,POPLATEK MESICNE,1996-02-21
4,5,15,POPLATEK MESICNE,1997-05-30


In [72]:
df = pd.merge(loan_df_, account_df_, how='left', on=['account_id'])
print(df.shape)
df.head()

(682, 10)


,loan_loan_id,account_id,loan_date,loan_amount,loan_duration,loan_payments,loan_status,acc_district_id,acc_frequency,acc_date
0,4959,2,1994-01-05,80952,24,3373.00,A,1,POPLATEK MESICNE,1993-02-26
1,4961,19,1996-04-29,30276,12,2523.00,B,21,POPLATEK MESICNE,1995-04-07
2,4962,25,1997-12-08,30276,12,2523.00,A,68,POPLATEK MESICNE,1996-07-28
3,4967,37,1998-10-14,318480,60,5308.00,D,20,POPLATEK MESICNE,1997-08-18
4,4968,38,1998-04-19,110736,48,2307.00,C,19,POPLATEK TYDNE,1997-08-08


## order manipulation

due to 1 account can have more order. So, we need to manipulate order values 

such as 

amount -> min, max, sum, mean, count

k_symbol -> onehot -> sum

In [73]:
order_df.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,


order_df there are "spacing". so, we add NaN.

In [74]:
order_df['k_symbol'].value_counts()

SIPO        3502
            1379
UVER         717
POJISTNE     532
LEASING      341
Name: k_symbol, dtype: int64

In [75]:
order_df_ = order_df.replace("", np.nan)
order_df_['k_symbol'].value_counts()

SIPO        3502
UVER         717
POJISTNE     532
LEASING      341
Name: k_symbol, dtype: int64

In [76]:
order_df_.loc[order_df['k_symbol']=="POJISTNE", 'k_symbol'] = 'insurrance'
order_df_.loc[order_df['k_symbol']=="SIPO", 'k_symbol'] = 'household'
order_df_.loc[order_df['k_symbol']=="LEASING", 'k_symbol'] = 'leasing'
order_df_.loc[order_df['k_symbol']=="UVER", 'k_symbol'] = 'loan'

print(order_df_.shape)
order_df_.head()

(6471, 6)


,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,household
1,29402,2,ST,89597016,3372.7,loan
2,29403,2,QR,13943797,7266.0,household
3,29404,3,WX,83084338,1135.0,household
4,29405,3,CD,24485939,327.0,NaN


k_symbol onehot encoding

In [77]:
order_df_ = order_df_.join(pd.get_dummies(order_df_['k_symbol'], prefix='k_symbol_'))
order_df_.head()

,order_id,account_id,bank_to,account_to,amount,k_symbol,k_symbol__household,k_symbol__insurrance,k_symbol__leasing,k_symbol__loan
0,29401,1,YZ,87144583,2452.0,household,1,0,0,0
1,29402,2,ST,89597016,3372.7,loan,0,0,0,1
2,29403,2,QR,13943797,7266.0,household,1,0,0,0
3,29404,3,WX,83084338,1135.0,household,1,0,0,0
4,29405,3,CD,24485939,327.0,NaN,0,0,0,0


Drop bank_to, account_to, k_symbol

In [78]:
order_df_ = order_df_.drop(['order_id', 'bank_to', 'account_to', 'k_symbol'], axis=1)
order_df_.head()

,account_id,amount,k_symbol__household,k_symbol__insurrance,k_symbol__leasing,k_symbol__loan
0,1,2452.0,1,0,0,0
1,2,3372.7,0,0,0,1
2,2,7266.0,1,0,0,0
3,3,1135.0,1,0,0,0
4,3,327.0,0,0,0,0


In [ ]:
def summary_group_cate_data(main_df, add_df, col_names=list, how='left', on=['account_id'], validate='one_to_many', by="account_id"):
    t = pd.merge(main_df, add_df, how=how, on=on, validate=validate)
    t = t.groupby(by=by, as_index=False)[col_names].sum()

In [79]:
order_ksymbol = pd.merge(df, order_df_, how='left', on=['account_id'], validate='one_to_many')
order_ksymbol = order_ksymbol.groupby(by="account_id", as_index=False)[['k_symbol__household', 'k_symbol__insurrance', 'k_symbol__leasing', 'k_symbol__loan']].sum()
order_ksymbol

,account_id,k_symbol__household,k_symbol__insurrance,k_symbol__leasing,k_symbol__loan
0,2,1,0,0,1
1,19,0,0,0,1
2,25,1,1,0,1
3,37,1,1,0,1
4,38,1,1,0,1
...,...,...,...,...,...
677,11327,1,0,0,1
678,11328,0,0,0,1
679,11349,1,0,0,1
680,11359,1,0,0,1


In [199]:
def summary_group_num_data(main_df, add_df, col_name=str, how='left', on=['account_id'], validate='one_to_many', by="account_id"):
    t = pd.merge(main_df, add_df, how=how, on=on, validate=validate)
    tt = t.groupby(by=by)[col_name].agg([min, max, sum]).reset_index()

    tt['mean'] = t.groupby(by="account_id")[col_name].mean().reset_index(drop=True)
    tt['count'] = t.groupby(by="account_id")[col_name].count().reset_index(drop=True)

    return tt

In [200]:
order_amount_df = summary_group_num_data(df, order_df_, col_name='amount')

order_amount_df = add_prefix_to_colnames(order_amount_df, prefix='amount_')
order_amount_df

,account_id,amount_min,amount_max,amount_sum,amount_mean,amount_count
0,2,3372.7,7266.0,10638.7,5319.350000,2
1,19,2523.2,2523.2,2523.2,2523.200000,1
2,25,164.0,6817.0,10614.2,2653.550000,4
3,37,97.0,5307.5,10305.5,2576.375000,4
4,38,228.0,6901.0,9666.8,2416.700000,4
...,...,...,...,...,...,...
677,11327,1632.0,3693.0,5325.0,2662.500000,2
678,11328,4674.0,4674.0,4674.0,4674.000000,1
679,11349,6998.0,10886.0,17884.0,8942.000000,2
680,11359,254.0,4518.0,9274.3,3091.433333,3


In [81]:
order_amount_df = add_prefix_to_colnames(order_amount_df, prefix='order_')
order_ksymbol = add_prefix_to_colnames(order_ksymbol, prefix='order_')

In [82]:
df = pd.merge(df, order_amount_df, how='left', on=['account_id'])
df = pd.merge(df, order_ksymbol, how='left', on=['account_id'])
df

,loan_loan_id,account_id,loan_date,loan_amount,loan_duration,loan_payments,loan_status,acc_district_id,acc_frequency,acc_date,order_amount_min,order_amount_max,order_amount_sum,order_amount_mean,order_amount_count,order_k_symbol__household,order_k_symbol__insurrance,order_k_symbol__leasing,order_k_symbol__loan
0,4959,2,1994-01-05,80952,24,3373.00,A,1,POPLATEK MESICNE,1993-02-26,3372.7,7266.0,10638.7,5319.350000,2,1,0,0,1
1,4961,19,1996-04-29,30276,12,2523.00,B,21,POPLATEK MESICNE,1995-04-07,2523.2,2523.2,2523.2,2523.200000,1,0,0,0,1
2,4962,25,1997-12-08,30276,12,2523.00,A,68,POPLATEK MESICNE,1996-07-28,164.0,6817.0,10614.2,2653.550000,4,1,1,0,1
3,4967,37,1998-10-14,318480,60,5308.00,D,20,POPLATEK MESICNE,1997-08-18,97.0,5307.5,10305.5,2576.375000,4,1,1,0,1
4,4968,38,1998-04-19,110736,48,2307.00,C,19,POPLATEK TYDNE,1997-08-08,228.0,6901.0,9666.8,2416.700000,4,1,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,7294,11327,1998-09-27,39168,24,1632.00,C,7,POPLATEK MESICNE,1997-10-15,1632.0,3693.0,5325.0,2662.500000,2,1,0,0,1
678,7295,11328,1998-07-18,280440,60,4674.00,C,54,POPLATEK MESICNE,1996-11-05,4674.0,4674.0,4674.0,4674.000000,1,0,0,0,1
679,7304,11349,1995-10-29,419880,60,6998.00,C,1,POPLATEK TYDNE,1995-05-26,6998.0,10886.0,17884.0,8942.000000,2,1,0,0,1
680,7305,11359,1996-08-06,54024,12,4502.00,A,61,POPLATEK MESICNE,1994-10-01,254.0,4518.0,9274.3,3091.433333,3,1,0,0,1


## Transaction

In [119]:
summary_df(trans_df)


(1056320, 10)


,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,1,1,1995-03-24,PRIJEM,VKLAD,1000,1000,NaN,NaN,NaN
1,5,1,1995-04-13,PRIJEM,PREVOD Z UCTU,3679,4679,NaN,AB,41403269.0
2,6,1,1995-05-13,PRIJEM,PREVOD Z UCTU,3679,20977,NaN,AB,41403269.0
3,7,1,1995-06-13,PRIJEM,PREVOD Z UCTU,3679,26835,NaN,AB,41403269.0
4,8,1,1995-07-13,PRIJEM,PREVOD Z UCTU,3679,30415,NaN,AB,41403269.0


missing value


trans_id           0
account_id         0
date               0
type               0
operation     183114
amount             0
balance            0
k_symbol      535314
bank          782812
account       760931
dtype: int64

replace None (null values), spacing with NaN

In [120]:
# check values not in list (show spacing)
trans_df.loc[~trans_df['k_symbol'].isin(["POJISTNE", "SLUZBY", "UROK", "SANKC. UROK", "SIPO", "DUCHOD", "UVER", np.nan]) , :]

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account


In [121]:
# fill spacing with nan
trans_df = trans_df.replace(' ', np.nan)
# check
trans_df.loc[~trans_df['k_symbol'].isin(["POJISTNE", "SLUZBY", "UROK", "SANKC. UROK", "SIPO", "DUCHOD", "UVER", np.nan]) , :]

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account


In [122]:
# fill None with nan
trans_df = trans_df.fillna(value=np.nan)
trans_df

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account
0,1,1,1995-03-24,PRIJEM,VKLAD,1000,1000,NaN,NaN,NaN
1,5,1,1995-04-13,PRIJEM,PREVOD Z UCTU,3679,4679,NaN,AB,41403269.0
2,6,1,1995-05-13,PRIJEM,PREVOD Z UCTU,3679,20977,NaN,AB,41403269.0
3,7,1,1995-06-13,PRIJEM,PREVOD Z UCTU,3679,26835,NaN,AB,41403269.0
4,8,1,1995-07-13,PRIJEM,PREVOD Z UCTU,3679,30415,NaN,AB,41403269.0
...,...,...,...,...,...,...,...,...,...,...
1056315,3682983,10451,1998-08-31,PRIJEM,NaN,62,17300,UROK,NaN,NaN
1056316,3682984,10451,1998-09-30,PRIJEM,NaN,49,13442,UROK,NaN,NaN
1056317,3682985,10451,1998-10-31,PRIJEM,NaN,34,10118,UROK,NaN,NaN
1056318,3682986,10451,1998-11-30,PRIJEM,NaN,26,8398,UROK,NaN,NaN


type have class "PRIJEM", "VYDAJ". Also "VYBER" (not show in https://web.archive.org/web/20180506035658/http://lisp.vse.cz/pkdd99/Challenge/berka.htm)

include "VYBER" to "VYDAJ" as withdrawal.

In [128]:
trans_df['type'].value_counts()

VYDAJ     634571
PRIJEM    405083
VYBER      16666
Name: type, dtype: int64

In [130]:
trans_df['operation'].value_counts()

VYBER             434918
PREVOD NA UCET    208283
VKLAD             156743
PREVOD Z UCTU      65226
VYBER KARTOU        8036
Name: operation, dtype: int64

In [131]:
trans_df['k_symbol'].value_counts()

UROK           183114
SLUZBY         155832
SIPO           118065
DUCHOD          30338
POJISTNE        18500
UVER            13580
SANKC. UROK      1577
Name: k_symbol, dtype: int64

In [150]:
trans_df_ = trans_df.copy()
# type
trans_df_.loc[trans_df['type']=="PRIJEM", 'type'] = 'credit'
trans_df_.loc[trans_df['type']=="VYDAJ", 'type'] = 'withdrawal'
trans_df_.loc[trans_df['type']=="VYBER", 'type'] = 'withdrawal'

# operation
trans_df_.loc[trans_df['operation']=="VYBER KARTOU", 'operation'] = 'credit_card_withdrawal'
trans_df_.loc[trans_df['operation']=="VKLAD", 'operation'] = 'credit_in_cash'
trans_df_.loc[trans_df['operation']=="PREVOD Z UCTU", 'operation'] = 'collection_from_anotherbank'
trans_df_.loc[trans_df['operation']=="VYBER", 'operation'] = 'withdrawal_in_cash'
trans_df_.loc[trans_df['operation']=="PREVOD NA UCET", 'operation'] = 'remittance_to_anotherbank'

# k_symbol
trans_df_.loc[trans_df['k_symbol']=="POJISTNE", 'k_symbol'] = 'insurrance'
trans_df_.loc[trans_df['k_symbol']=="SLUZBY", 'k_symbol'] = 'payment_for_statement'
trans_df_.loc[trans_df['k_symbol']=="UROK", 'k_symbol'] = 'interest_credited'
trans_df_.loc[trans_df['k_symbol']=="SANKC. UROK", 'k_symbol'] = 'sanction_interest_neg_bal'
trans_df_.loc[trans_df['k_symbol']=="SIPO", 'k_symbol'] = 'household'
trans_df_.loc[trans_df['k_symbol']=="DUCHOD", 'k_symbol'] = 'old-age_pension'
trans_df_.loc[trans_df['k_symbol']=="UVER", 'k_symbol'] = 'loan'

# onehot
trans_df_ = trans_df_.join(pd.get_dummies(trans_df_['type'], prefix='type'))
trans_df_ = trans_df_.join(pd.get_dummies(trans_df_['operation'], prefix='operation'))
trans_df_ = trans_df_.join(pd.get_dummies(trans_df_['k_symbol'], prefix='k_symbol'))

trans_df_.head()

,trans_id,account_id,date,type,operation,amount,balance,k_symbol,bank,account,...,operation_credit_in_cash,operation_remittance_to_anotherbank,operation_withdrawal_in_cash,k_symbol_household,k_symbol_insurrance,k_symbol_interest_credited,k_symbol_loan,k_symbol_old-age_pension,k_symbol_payment_for_statement,k_symbol_sanction_interest_neg_bal
0,1,1,1995-03-24,credit,credit_in_cash,1000,1000,NaN,NaN,NaN,...,1,0,0,0,0,0,0,0,0,0
1,5,1,1995-04-13,credit,collection_from_anotherbank,3679,4679,NaN,AB,41403269.0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1995-05-13,credit,collection_from_anotherbank,3679,20977,NaN,AB,41403269.0,...,0,0,0,0,0,0,0,0,0,0
3,7,1,1995-06-13,credit,collection_from_anotherbank,3679,26835,NaN,AB,41403269.0,...,0,0,0,0,0,0,0,0,0,0
4,8,1,1995-07-13,credit,collection_from_anotherbank,3679,30415,NaN,AB,41403269.0,...,0,0,0,0,0,0,0,0,0,0


In [151]:
trans_df_.columns

Index(['trans_id', 'account_id', 'date', 'type', 'operation', 'amount',
       'balance', 'k_symbol', 'bank', 'account', 'type_credit',
       'type_withdrawal', 'operation_collection_from_anotherbank',
       'operation_credit_card_withdrawal', 'operation_credit_in_cash',
       'operation_remittance_to_anotherbank', 'operation_withdrawal_in_cash',
       'k_symbol_household', 'k_symbol_insurrance',
       'k_symbol_interest_credited', 'k_symbol_loan',
       'k_symbol_old-age_pension', 'k_symbol_payment_for_statement',
       'k_symbol_sanction_interest_neg_bal'],
      dtype='object')

In [157]:
trans_df_drop = trans_df_.drop(['trans_id', 'type', 'operation', 'k_symbol', 'bank', 'account'], axis=1)
trans_df_drop.head()

,account_id,date,amount,balance,type_credit,type_withdrawal,operation_collection_from_anotherbank,operation_credit_card_withdrawal,operation_credit_in_cash,operation_remittance_to_anotherbank,operation_withdrawal_in_cash,k_symbol_household,k_symbol_insurrance,k_symbol_interest_credited,k_symbol_loan,k_symbol_old-age_pension,k_symbol_payment_for_statement,k_symbol_sanction_interest_neg_bal
0,1,1995-03-24,1000,1000,1,0,0,0,1,0,0,0,0,0,0,0,0,0
1,1,1995-04-13,3679,4679,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1,1995-05-13,3679,20977,1,0,1,0,0,0,0,0,0,0,0,0,0,0
3,1,1995-06-13,3679,26835,1,0,1,0,0,0,0,0,0,0,0,0,0,0
4,1,1995-07-13,3679,30415,1,0,1,0,0,0,0,0,0,0,0,0,0,0


date data -> 

find max, min date in each account_id and find duration. บอกระยะเวลาที่ใช้บัตรทำ transaction ตั้งแต่แรกยันล่าสุด

In [186]:
trans_df_drop['date'] = pd.to_datetime(trans_df_drop['date'], format='%Y-%m-%d')
trans_date = trans_df_drop.groupby(by='account_id').agg([min, max])['date']

trans_date['duration'] = trans_date['max'] - trans_date['min']
trans_date = add_prefix_to_colnames(trans_date, prefix='date_')

trans_date.head()

,date_min,date_max,date_duration
account_id,,,
1,1995-03-24,1998-12-31,1378 days
2,1993-02-26,1998-12-31,2134 days
3,1997-07-07,1998-12-31,542 days
4,1996-02-21,1998-12-31,1044 days
5,1997-05-30,1998-12-31,580 days


Numerical data 

amount, balance -> min, max, sum, mean, count

In [189]:
trans_df_ = trans_df_drop.copy()
trans_df_.head()

,account_id,date,amount,balance,type_credit,type_withdrawal,operation_collection_from_anotherbank,operation_credit_card_withdrawal,operation_credit_in_cash,operation_remittance_to_anotherbank,operation_withdrawal_in_cash,k_symbol_household,k_symbol_insurrance,k_symbol_interest_credited,k_symbol_loan,k_symbol_old-age_pension,k_symbol_payment_for_statement,k_symbol_sanction_interest_neg_bal
0,1,1995-03-24,1000,1000,1,0,0,0,1,0,0,0,0,0,0,0,0,0
1,1,1995-04-13,3679,4679,1,0,1,0,0,0,0,0,0,0,0,0,0,0
2,1,1995-05-13,3679,20977,1,0,1,0,0,0,0,0,0,0,0,0,0,0
3,1,1995-06-13,3679,26835,1,0,1,0,0,0,0,0,0,0,0,0,0,0
4,1,1995-07-13,3679,30415,1,0,1,0,0,0,0,0,0,0,0,0,0,0


In [187]:
nume_cols = ['amount', 'balance']

Cateforical data

type, operation, k_symbol -> onehot -> sum

In [ ]:
trans_cate_data = pd.merge(df, trans_df_drop, how='left', on=['account_id'], validate='one_to_many')

trans_cate_data.groupby(by="account_id", as_index=False)[['k_symbol__household', 'k_symbol__insurrance', 'k_symbol__leasing', 'k_symbol__loan']].sum()

In [ ]:
order_ksymbol = pd.merge(df, trans_df_drop, how='left', on=['account_id'], validate='one_to_many')
order_ksymbol = order_ksymbol.groupby(by="account_id", as_index=False)[['k_symbol__household', 'k_symbol__insurrance', 'k_symbol__leasing', 'k_symbol__loan']].sum()
order_ksymbol

In [ ]:
df = pd.merge(df, trans_df_, how='left', on=['account_id'])
df

In [ ]:
trans_df_final = add_prefix_to_colnames(trans_df_drop, prefix='order_')
trans_df_final